In [1]:
import sys
#sys.path.insert(0, "..")
sys.path.insert(0, "../..")

In [2]:
from smg.datasets.ldp_5_cleansed import LDP5Cleansed
from smg.models.recurrent import RecurrentSMG

In [3]:
from smg.experiments.train_lstm import ex

# Train modell

In [10]:
ex.run('print_config')

INFO - train_lstm - Running command 'print_config'
INFO - train_lstm - Started
INFO - train_lstm - Completed after 0:00:00


Configuration (modified, added, typechanged, doc):
  BEATS_PER_MEASURE = 4
  batch_size = 8                     # data_loader config
  beat_resolution = 4
  data_dir = '../data/few_examples'  # #"../../../data/lpd_5",
  hidden_size = 200                  # model configs
  lowest_pitch = 24
  lr = 0.01
  measures_per_sample = 4
  n_instruments = 5                  # general configs
  n_pitches = 72
  n_workers = 0
  num_epochs = 10                    # train configs
  num_layers = 3
  seed = 863668994                   # the random seed for this experiment
  timesteps_per_measure = 16


In [12]:
config_updates = {
    "data_dir": "../../../data/few_examples"
}
ex.run(config_updates=config_updates)

WARNING - train_lstm - No observers have been added to this run
INFO - train_lstm - Running command 'main'
INFO - train_lstm - Started
   1. epoch:   0%|                                                                               | 0/4 [00:00<?, ?it/s]

Loaded 37 samples!


  10. epoch: 100%|██████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.52it/s, loss=11]
INFO - train_lstm - Completed after 0:00:10


## Generate Music

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
import pypianoroll as pp
import torch.optim as optim

from glob import glob

from tqdm import tqdm

In [ ]:
# use model and generate a fuckn mid

# how you ask?
# well good question! let me tell you how

# start with a piano roll sequence
# should have dimension (batch_size, n_instruments, seq_length, n_pitches)
# batch size is 1, n_instrument is 5, n_pitches is 72
from torch.distributions.multinomial import Multinomial
from datetime import datetime


def generate_pianoroll(pianoroll, beat_resolution, lowest_pitch, n_pitches):
    '''
    :param pianoroll: uint8 array, shape = (instruments, time, 128) - min value = 0, max value = 127
    :param beat_resolution: don't touch this unless you know what you are doing
    :return: a multi track piano roll
    '''
    
    INSTRUMENTS = ['Drums', 'Piano', 'Guitar', 'Bass', 'Strings']
    INSTRUMENT_MIDI_METADATA = {
        0: (0, True),
        1: (0, False),
        2: (25, False),
        3: (33, False),
        4: (48, False)
    }

    tempo = 120.0
    downbeat=None

    assert(np.all(np.logical_and(pianoroll >= 0, pianoroll < 128)))
    pianoroll = pianoroll.astype(np.uint8)

    full_pitch_pianoroll = np.zeros((pianoroll.shape[0], pianoroll.shape[1], 128))
    full_pitch_pianoroll[:, :, lowest_pitch:lowest_pitch + n_pitches] = pianoroll

    # create tracks
    tracks = []
    for i in range(len(full_pitch_pianoroll)):
        program, is_drum = INSTRUMENT_MIDI_METADATA[i]
        tracks.append(
            pp.Track(
                full_pitch_pianoroll[i],
                is_drum=is_drum,
                program=program,
                name=INSTRUMENTS[i]
            )
        )

    # create multitrack piano rolls
    return pp.Multitrack(
        tracks=tracks,
        tempo=tempo * (np.ones(full_pitch_pianoroll.shape[1])),
        downbeat=downbeat,
        beat_resolution=beat_resolution,
        name='generated'
    )


def rand_stub_generator(seq_len):
    n_pitches = 72
    n_instruments = 5
    
    m = Multinomial(total_count=1, probs=torch.ones(1, n_instruments, seq_len, n_pitches))
    
    return lambda : m.sample()


def generate_mid(stub, model, n_iter=10, seq_length=None):
    if seq_length is None:
        seq_length = stub.size(2)
    
    pianoroll = stub
    
    for _ in range(n_iter):
        # generate next timestep
        y_hat = model.forward(stub)
        
        # add timestep to pianoroll
        pianoroll = torch.cat([pianoroll, y_hat], dim=2)
        # slide stub
        stub = pianoroll[:, :, -seq_len:, :]
        

    # convert pianoroll -> pp.Multitrack -> .mid file
    pianoroll_view = pianoroll.view(-1, pianoroll.size(3))
    max_p, argmax_p = pianoroll_view.max(dim=-1)
    
    new_roll = torch.zeros_like(pianoroll_view)
    new_roll[np.arange(new_roll.size(0)), argmax_p.type(torch.LongTensor)] = max_p
    
    pianoroll = new_roll.view(pianoroll.size())
    pianoroll = pianoroll.detach()[0].numpy() * 127
    
    pianoroll[np.logical_and(pianoroll > 1, pianoroll < 60)] += 60

    return generate_pianoroll(pianoroll, beat_resolution=4, lowest_pitch=24, n_pitches=72)

    
def stub_from_file(file, seq_length, random_start=True):
    lowest_pitch = 24
    n_pitches = 72
    n_instruments = 5
    beat_resolution = 4
    
    multitrack_roll = pp.load(file)
    multitrack_roll.downsample(multitrack_roll.beat_resolution // beat_resolution)
    
    stacked = multitrack_roll.get_stacked_pianoroll().transpose(2, 0, 1)
    
    start = 0 # np.random.choice(stacked.shape[1] - seq_length)
    if random_start:
        start = np.random.choice(stacked.shape[1] - seq_length)
     
    print(start)
    stub = stacked[:, start:start+seq_length, lowest_pitch:lowest_pitch+n_pitches]
    
    return torch.from_numpy(stub).type(dtype=torch.FloatTensor).unsqueeze(0) / 127

In [ ]:
model.load_state_dict(torch.load('lstm_finished.pth', map_location=dev))

In [ ]:
data_dir = ds.data_dir
seq_len = 16*4

In [ ]:
gen = rand_stub_generator(seq_len)
stub = gen()
sample = generate_mid(stub, model)

In [ ]:
import pypianoroll.plot as ppplt

In [ ]:
files = glob(data_dir + "/**/*.npz", recursive=True)

In [ ]:
file = np.random.choice(files)
#file = files[]

print(file)
stub = stub_from_file(file, seq_len)
sample = generate_mid(stub, model, 10)
sample.write("../../../out/out_{}.mid".format(int(datetime.timestamp(datetime. now()))))

#sample.write('out.mid')
fig, axs = ppplt(sample.tracks[0])

fig.set_size_inches(18.5, 10.5)
plt.show()